In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelBinarizer, scale
from sklearn_pandas import DataFrameMapper
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle

In [2]:
from matplotlib.colors import LogNorm

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [4]:
import os
from sys import argv
import pdb

In [5]:
def m(x, w):
    """Weighted Mean"""
    return np.sum(x * w) / np.sum(w)

def cov(x, y, w):
    """Weighted Covariance"""
    return np.sum(w * (x - m(x, w)) * (y - m(y, w))) / np.sum(w)

def corr(x, y, w):
    """Weighted Correlation"""
    return cov(x, y, w) / np.sqrt(cov(x, x, w) * cov(y, y, w))

In [6]:
def get_data(data_dir):
    fs = [data_dir + f for f in os.listdir(data_dir) if ('signal' in f or 'WZ' in f) and f[0] != '.']
    df = pd.DataFrame()

    for f in fs:
        print f
        new_df = pd.read_csv(f)
        df = pd.concat([df, new_df], ignore_index = True)
        df.index = range(len(df))

    return df

In [7]:
def add_cl_ix(df):
    df['is_sig'] = [1 if 'signal' in val else 0 for val in df.cl.values]
    return df

In [8]:
class WWdataset(Dataset):

    def __init__(self, pd_dataset, non_input=None):
        self.dataset = pd_dataset
        
        self.scale_dataset()
        
        self.non_input = ['runNumber', 'lbNumber', 'eventNumber', 'SFOS', 'is_sig', 'weight', 'cl', 'preds']
        self.non_input += non_input
        self.input_vars = [col for col in self.dataset.columns if not col in self.non_input]

        self.target_var = ['is_sig']
        self.weight_var = ['weight']
        
        self.input_np = self.dataset[self.input_vars].as_matrix().astype(dtype=np.float32)
        self.target_np = self.dataset[self.target_var].as_matrix().astype(dtype=int)
        self.weight_np =self.dataset[self.weight_var].as_matrix().astype(dtype=np.float32)

        self.inputs = torch.from_numpy(self.input_np)
        self.target = torch.from_numpy(self.target_np)
        self.weight = torch.from_numpy(self.weight_np)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        inputs = self.inputs[idx]
        target = self.target[idx]
        weight = self.weight[idx]
        return inputs, target, weight

    def n_input(self):
        return len(self.input_vars)
    
    def scale_dataset(self):
        continuous_cols = [column for column in self.dataset.columns 
                           if self.dataset[column].dtype is np.dtype('float64') and column != 'weight']
        self.dataset[continuous_cols] = scale(self.dataset[continuous_cols])
        

In [9]:
def net_logistic_regression(n_input):
    model = torch.nn.Sequential(
        torch.nn.Linear(n_input, 2),
    )
    return model

In [10]:
def net_deep_logistic_regression(n_input):
    model = torch.nn.Sequential(
        torch.nn.Linear(n_input, 200),
        torch.nn.ReLU(),
        torch.nn.Linear(200, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 10),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 2),
    )
    return model

In [ ]:
data_dir = "../../data/"
pandas_dataset = add_cl_ix(get_data(data_dir))

In [ ]:
non_input = ['l0_m', 'l1_m', 'l2_m', 'SF0S']
train_fraction = 0.7
pandas_dataset = shuffle(pandas_dataset)
n_switch = int(0.7 * len(pandas_dataset))

train_dataset = pandas_dataset[:n_switch]
test_dataset = pandas_dataset[n_switch:]

train_dataset = WWdataset(train_dataset)
test_dataset = WWdataset(test_dataset)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=200, shuffle=True, num_workers=2)

In [ ]:
#net = net_logistic_regression(in_dataset.n_input())
net = net_deep_logistic_regression(in_dataset.n_input())
criterion = nn.CrossEntropyLoss(reduce=False)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5)

# Training

In [ ]:
def compute_loss_batch(batch, net):
    inputs, label, weight = batch
    inputs, label, weight = Variable(inputs), Variable(label), Variable(weight)

    output = net(inputs)
    losses = criterion(output, label.squeeze())
    loss = (losses * weight.squeeze()).sum()

    return loss

def compute_loss_epoch(loader, net):
    loss = 0.
    for i, batch in enumerate(loader):
        loss += compute_loss_batch(batch, net).data[0]
    loss /= len(loader)
    return loss

In [ ]:
for epoch in range(6):
        print
        print "epoch: ", epoch
        running_loss = 0.
        for i, batch in enumerate(trainloader):

            optimizer.zero_grad()
            loss = compute_loss_batch(batch, net)
            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]
            # if i % 200 == 199:    # print every 200 mini-batches
            #   print "batch:  {}, loss: {}".format(i+1, running_loss/(i+1))
        print "running loss: ", running_loss/len(trainloader)
        print "train loss: ", compute_loss_epoch(trainloader, net)        
        print "test loss: ", compute_loss_epoch(testloader, net)

In [ ]:
for epoch in range(6):
        print
        print "epoch: ", epoch
        running_loss = 0.
        for i, data in enumerate(trainloader):
            inputs, label, weight = data
            inputs, label, weight = Variable(inputs), Variable(label), Variable(weight)

            optimizer.zero_grad()
            output = net(inputs)
            losses = criterion(output, label.squeeze())
            loss = (losses * weight.squeeze()).sum()
            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]
            if i % 200 == 199:    # print every 200 mini-batches
                print "batch:  {}, loss: {}".format(i+1, running_loss/(i+1))

# Prediction and Signficance

In [ ]:
input_for_pred = Variable(in_dataset.inputs)
predicted_scores = net(input_for_pred)
predicted_prob = nn.functional.softmax(predicted_scores, dim=1)

In [ ]:
pandas_dataset['preds'] = predicted_prob.data.numpy()[:,1]

## ROC curve

In [ ]:
df = pandas_dataset

fpr, tpr, thresholds = metrics.roc_curve(df.is_sig.values, df.preds.values, pos_label=1)
fig = plt.figure(figsize=(6,6))
plt.plot(fpr, tpr)
plt.title('NN Regression ROC')
plt.annotate('Area: ' + str(round(metrics.auc(fpr, tpr), 2)), xy=(.8,.2), xycoords='axes fraction')
plt.xlim((0,1))
plt.ylim((0,1))
plt.plot([0,1], [0,1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.show
plt.savefig("./plots/roc_curve_train_nn_regression.pdf")

## Compute signal, background and significance

In [ ]:
def n_sig_n_bkg(df, SF0S, n_bins, plot):
    df = df[df['SFOS'] == SF0S] 

    x_bins = np.linspace(0, 1, n_bins)
    fig = plt.figure(figsize=(10,6))
    gs = gridspec.GridSpec(3, 1)
    ax = plt.subplot(gs[0,:])
    plt.title('NN Regression Separability on train sample, SF0S = {}'.format(SF0S))
    # n_bkg,bins,paint = plt.hist(df[df.is_sig == 0].preds, bins=x_bins, weights=df[df.is_sig == 0].weight, color='r')
    n_bkg,bins,paint = plt.hist(df[df.is_sig == 0].preds.as_matrix(), bins=x_bins, weights=df[df.is_sig == 0].weight.as_matrix(), color='r')
    plt.xlim([0, 1])
    plt.yscale('log')
    plt.ylabel(r'Weighted Background Counts', size=9)
    plt.legend(handles=[mpatches.Patch(color='red', label='Background')])
    ax1 = plt.subplot(gs[1,:])
    n_sig,bins,paint = plt.hist(df[df.is_sig == 1].preds.as_matrix(), bins=x_bins, weights=df[df.is_sig == 1].weight.as_matrix(), color='g')
    plt.xlim([0, 1])
    plt.yscale('log')
    plt.ylabel(r'Weighted Signal Counts', size=9)
    plt.legend(handles=[mpatches.Patch(color='green', label='Signal')])
    ax2 = plt.subplot(gs[2,:])
    plt.bar((x_bins[:-1] + x_bins[1:]) / 2., n_sig / np.sqrt(n_bkg), width=x_bins[1] - x_bins[0], color='k')
    plt.xlim([0, 1])
    plt.ylabel(r'Significance ($S/\sqrt{B})$', size=9)
    plt.xlabel('Probability Event is a Signal')

    plt.tight_layout()
    if plot==True:
        plt.show()
    return n_sig, n_bkg

In [ ]:
sign_combined = 0.
for SF... in range(3):
    n_sig, n_bkg = n_sig_n_bkg(pandas_dataset, SF0S, n_bins=20, plot=True)
    
    total_significance = 0.
    for s, b in zip(n_sig, n_bkg):
        if b == 0:
            continue
        else:
            total_significance += s**2/b
    sign_combined += total_significance
    total_significance = np.sqrt(total_significance)
    print "signficance with SF0S = {} : {} sigma".format(SF0S, total_significance)

print "combined significance: {}".format(np.sqrt(sign_combined))

In [ ]:
significance = []
signal = []
background = []
probability_cuts = np.linspace(0, 1, 20)
for SF0S in range(3):
    significance.append([])
    signal.append([])
    background.append([])
    for p in probability_cuts:
        sig = pandas_dataset[(pandas_dataset['SFOS'] == SF0S)
                                 & (pandas_dataset['preds'] > p)
                                 & (pandas_dataset['is_sig'] == 1)]['weight'].sum()
        bg = pandas_dataset[(pandas_dataset['SFOS'] == SF0S)
                                 & (pandas_dataset['preds'] > p)
                                 & (pandas_dataset['is_sig'] == 0)]['weight'].sum()
        
        # significance[SF0S].append(sig/np.sqrt(bg))
        significance[SF0S].append( np.sqrt(2*( (sig + bg)*np.log(1+sig/bg) -sig)) )
        
        signal[SF0S].append(sig)
        background[SF0S].append(bg)
    plt.plot(probability_cuts, significance[SF0S], label="SF0S = {}".format(SF0S))

plt.legend(loc='best')
plt.title('Significance varying cuts and for various SF0S')
plt.xlabel('Classifier output')
plt.ylabel('Signficance')
plt.xlim((0,1))
plt.grid()
plt.show()

In [ ]:
for SF0S in range(3):
    plt.plot(probability_cuts, background[SF0S], label="SF0S = {}".format(SF0S))

plt.legend(loc='best')
plt.title('Number of background events varying cuts and for various SF0S')
plt.xlabel('Classifier output')
plt.ylabel('Number of background events')
plt.xlim((0,1))
plt.yscale('log')
plt.grid()
plt.show()

In [ ]:
for SF0S in range(3):
    plt.plot(probability_cuts, signal[SF0S], label="SF0S = {}".format(SF0S))

plt.legend(loc='best')
plt.title('Number of signal events varying cuts and for various SF0S')
plt.xlabel('Classifier output')
plt.ylabel('Number of signal events')
plt.xlim((0,1))
plt.yscale('log')
plt.grid()
plt.show()

# Testing plots

In [ ]:
figures = []
for var in in_dataset.input_vars:
    colors = ['r', 'b']
    title = {0: 'background', 1: 'signal'}
    
    f, axes = plt.subplots(1, 2, sharey=True)
    for sig in [0, 1]:        
        x = pandas_dataset[pandas_dataset['is_sig'] == sig][var]
        y = pandas_dataset[pandas_dataset['is_sig'] == sig]['preds']
        weight = pandas_dataset[pandas_dataset['is_sig'] == sig]['weight']
        
        h = axes[sig].hist2d(x, y, weights=weight, bins=20, norm=LogNorm())
        axes[sig].set_ylabel('classifier output')
        axes[sig].set_xlabel(var)
        axes[sig].set_title(title[sig] + ', corr: {:.2f}'.format(corr(x, y, weight)))
    #f.colorbar(h[3]) 
    # plt.show()
    figures.append(f)

import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("2D_plots.pdf")
for fig in figures:
    pdf.savefig( fig )
pdf.close()

# Various trials

In [ ]:
df = pandas_dataset
df = df[df['SFOS'] == 0] 

x_bins = np.linspace(0, max(df.preds), 30)
fig = plt.figure(figsize=(10,6))
gs = gridspec.GridSpec(3, 1)
ax = plt.subplot(gs[0,:])
plt.title('NN Regression Separability on train sample')
# n_bkg,bins,paint = plt.hist(df[df.is_sig == 0].preds, bins=x_bins, weights=df[df.is_sig == 0].weight, color='r')
n_bkg,bins,paint = plt.hist(df[df.is_sig == 0].preds.as_matrix(), bins=x_bins, weights=df[df.is_sig == 0].weight.as_matrix(), color='r')
plt.yscale('log')
plt.ylabel(r'Weighted Background Counts', size=9)
plt.legend(handles=[mpatches.Patch(color='red', label='Background')])
ax1 = plt.subplot(gs[1,:])
n_sig,bins,paint = plt.hist(df[df.is_sig == 1].preds.as_matrix(), bins=x_bins, weights=df[df.is_sig == 1].weight.as_matrix(), color='g')
plt.yscale('log')
plt.ylabel(r'Weighted Signal Counts', size=9)
plt.legend(handles=[mpatches.Patch(color='green', label='Signal')])
ax2 = plt.subplot(gs[2,:])
plt.bar((x_bins[:-1] + x_bins[1:]) / 2., n_sig / np.sqrt(n_bkg), width=x_bins[1] - x_bins[0], color='k')
plt.ylabel(r'Significance ($S/\sqrt{B})$', size=9)
plt.xlabel('Probability Event is a Signal')

plt.tight_layout()
plt.show()
# plt.savefig("./plots/preds_train_nn_regression.pdf")